In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS = 15  # эпох на обучение.
BATCH_SIZE = 40  # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR = 1e-4
VAL_SPLIT = 0.15  # сколько данных выделяем на тест = 15%

CLASS_NUM = 10  # количество классов в нашей задаче
IMG_SIZE = 224  # какого размера подаем изображения в сеть
IMG_CHANNELS = 3   # у RGB 3 канала
input_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/"  # рабочая директория

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH, exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts().plot(kind='barh')
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip, "r") as z:
        z.extractall(PATH)

print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12, 8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3, 3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

### Аугментация данных

In [ ]:
# train_datagen = ImageDataGenerator(
#rescale=1. / 255,
#rotation_range = 5,
# width_shift_range=0.1,
# height_shift_range=0.1,
# validation_split=VAL_SPLIT, # set validation split
# horizontal_flip=False)

test_datagen = ImageDataGenerator(
    rescale=1. / 255)

In [ ]:
# попробуем более продвинутый вариант аугментации данных
# все делаем здесь для удобства проверки
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor
    
from ImageDataAugmentor.image_data_augmentor import *
import albumentations


# сделаем более продвинутый генератор для train
AUGMENTATION = albumentations.Compose(
[
    # повороты
    albumentations.HorizontalFlip(p=0.25),
    albumentations.ElasticTransform(p=0.25),
    # цветовая аугментация
    albumentations.OneOf([
        albumentations.HueSaturationValue(p=1.),
        albumentations.RandomBrightnessContrast(p=1.),
        albumentations.RGBShift(p=1.)
    ], p=0.25),
    # качество изображения
    albumentations.OneOf([
        albumentations.GaussNoise(p=1.,var_limit=(10.0, 50.0)),
        albumentations.MultiplicativeNoise(p=1.),
        albumentations.JpegCompression(p=1.),
        albumentations.Downscale(scale_min=0.5,scale_max=0.99, p=1),
    ], p=0.5)])

train_datagen = ImageDataAugmentor(
        rescale=1./255,
        validation_split=VAL_SPLIT,
        augment = AUGMENTATION
        )

### Генерация данных

In [ ]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

# Обратите внимание, что для сабмита мы используем другой источник test_datagen.flow_from_dataframe. Как вы думаете, почему?

In [ ]:
# посмотрим на примеры работы генератора
x,y = train_generator.next()
print('Пример картинок из test_generator')
plt.figure(figsize=(12,8))

for i in range(0,9):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
plt.show()

# Построение модели

### Загружаем предобученную сеть FixEfficientNet-B7:
не будем выносить вверх все установки и импорты  для удобства поиска проверяющего 

In [ ]:
# предложенная автором модель. Пока ее не используем

#base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

# Устанавливаем новую "голову" (head)

#x = base_model.output
#x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(256, activation='relu')(x)
#x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
#predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
#model = Model(inputs=base_model.input, outputs=predictions)
#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
# модель EfficientNetB7

!pip install -q efficientnet
import efficientnet.tfkeras as efn

base_model = efn.EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# посмотрим на количество слоев в base_model
print(len(base_model.layers))

## Другие модели
### Вариант 1
#### EfficientNetB7 без обучения  ее слоев (Голову оставим предложенную автором)

In [ ]:
#base_model.trainable = False
#model=M.Sequential()
#model.add(base_model)
#model.add(L.GlobalAveragePooling2D())
#model.add(L.Dense(256, activation='relu'))
#model.add(L.Dropout(0.25))
#model.add(L.Dense(CLASS_NUM, activation='softmax'))


#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
#model.summary()

 Остановили обучение на:\
 train-loss: 1.3527 - accuracy: 0.5326 - val_loss: 1.1166 - val_accuracy: 0.6299


### Вариант 2 (с Finetuning)
#### EfficientNetB7 c заморозкой половины слоев (Голову оставим предложенную автором)

In [ ]:
#base_model.trainable = True

#fine_tuning_at= len(base_model.layers)//2

# заморозим первые слои
#for layer in base_model.layers[:fine_tuning_at]:
  #layer.trainable =  False

# проверим как все сработало:
#for layer in base_model.layers:
    #print(layer, layer.trainable)

In [ ]:
#model=M.Sequential()
#model.add(base_model)
#model.add(L.GlobalAveragePooling2D())
#model.add(L.Dense(256, activation='relu'))
#model.add(L.Dropout(0.25))
#model.add(L.Dense(CLASS_NUM, activation='softmax'))


#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
#model.summary()

### Итог
37/37 [==============================] - 54s 1s/step - loss: 0.2164 - accuracy: 0.9382
Accuracy: 93.82%

### Вариант 3 (с Finetuning)
#### EfficientNetB7 c заморозкой 199 слоев (Голову оставим предложенную автором)
#### также уменьшим размер батча до 40

In [ ]:
#base_model.trainable = True

#fine_tuning_at= 200

# заморозим первые слои
#for layer in base_model.layers[:fine_tuning_at]:
  #layer.trainable =  False

In [ ]:
#model=M.Sequential()
#model.add(base_model)
#model.add(L.GlobalAveragePooling2D())
#model.add(L.Dense(256, activation='relu'))
#model.add(L.Dropout(0.25))
#model.add(L.Dense(CLASS_NUM, activation='softmax'))


#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
#model.summary()

### Итог
59/59 [==============================] - 59s 986ms/step - loss: 0.2109 - accuracy: 0.9446
Accuracy: 94.46%

### Вариант 4 (с Finetuning)
#### EfficientNetB7 c заморозкой 199 слоев (Голову оставим предложенную автором)
#### также уменьшим размер батча до 40
#### добавим BatchNormalization

In [ ]:
base_model.trainable = True

fine_tuning_at= 200

# заморозим первые слои
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable =  False
    
model=M.Sequential()
model.add(base_model)
model.add(L.GlobalAveragePooling2D())
model.add(L.Dense(256, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.25))
model.add(L.Dense(CLASS_NUM, activation='softmax'))


model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

### Итог
59/59 [==============================] - 56s 946ms/step - loss: 0.2039 - accuracy: 0.9459
Accuracy: 94.59%

### Вариант 5 (с Finetuning) (НЕ удалось получить прироста, скорее наоборот стало все гораздо хуже. Будем использовать модель 4)
#### EfficientNetB7 c заморозкой 199 слоев (Голову оставим предложенную автором)
#### также уменьшим размер батча до 40
#### добавим BatchNormalization
#### Попробуем поуправлять Learning rate


In [ ]:
#base_model.trainable = True

#fine_tuning_at= 200

# заморозим первые слои
#for layer in base_model.layers[:fine_tuning_at]:
  #layer.trainable =  False
    
#model=M.Sequential()
#model.add(base_model)
#model.add(L.GlobalAveragePooling2D())
#model.add(L.Dense(256, activation='relu'))
#model.add(L.BatchNormalization())
#model.add(L.Dropout(0.25))
#model.add(L.Dense(CLASS_NUM, activation='softmax'))


#model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(), metrics=["accuracy"])

**Добавим LearningRateScheduler**

In [ ]:
# !git clone https://github.com/bckenstler/CLR.git 

# from CLR.clr_callback import CyclicLR

In [ ]:
# clr_step_size = int(8 * (len(train_generator)/BATCH_SIZE))
# base_lr = 1e-4
# max_lr = 1e-2
# mode='triangular'

# clr = CyclicLR(base_lr=base_lr, max_lr=max_lr, step_size=clr_step_size, mode=mode)

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
# добавим также earlystop 
earlystopping=EarlyStopping(monitor="val_accuracy", patience=5,restore_best_weights=True)
callbacks_list = [checkpoint,earlystopping]

# попытка добавить цикличный learning rate
#callbacks_list = [checkpoint,earlystopping,clr]

Обучаем:

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# Предсказание на тестовых данных

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

# Рекомендация: попробуйте добавить Test Time Augmentation (TTA)
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [ ]:
submission.head()

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

# Выводы:
Мы смогли увеличить результат модели с помощью:
1. Продвинутой аугментации данных
2. Выбора другой модели с использованием Finetuning и батч нормализации
3. Увеличения количества эпох на обучение
4. Манипуляции с learning rate не помогли, но возможно была использована неправильная реализация